In [2]:
import os 
from openbabel import pybel
from vina import Vina
os.chdir('/home/brightmoon/Experimental Class/3CL Protease/Virtual_Screen')

### Ligand_Prep

In [3]:
# smiles='CNC(=O)c1cc(cc(c1N[C@@H]2CCCC[C@@H]2NC(=O)c3cncc4c3cccc4)N(=O)=O)Br'
mol_id = 'may1'

In [4]:
# out=pybel.Outputfile(filename=f'./{mol_id}.mol2',format='mol2',overwrite=True)
# mol=pybel.readstring(string=smiles,format='smiles')
# mol.title=f'ligand_{smiles}'
# mol.make3D('mmff94s')
# mol.localopt(forcefield='mmff94s', steps=500)
# out.write(mol)
# out.close()

In [5]:
# if the ligand endwith pdb: from pdb to mol2
mol = pybel.readfile('pdb', f'./{mol_id}.pdb').__next__()
mol.write('mol2', f'./{mol_id}.mol2', overwrite=True)

### Protein_Perp

In [6]:
# 以1iep.pdb 为例，pdb-tools可以很好地处理包含杂原子、水分子的pdb文件
#os.system('pdb_selchain -A 7en8.pdb | pdb_delresname -HOH | pdb_delhetatm | pdb_tidy >1iep_clean.pdb')

In [10]:
os.system('prepare_receptor -r 7en8_clean.pdb -A "hydrogens" -o 7en8_receptor.pdbqt')

adding gasteiger charges to peptide


0

In [11]:
def get_vina_box(pdbqt_file):
    """TODO: Implement function to calculate the docking box parameters for AutoDock Vina."""
    pass

# return center = [x, y, z]

In [13]:
os.system(f'prepare_ligand -l {mol_id}.mol2 -o {mol_id}.pdbqt')

v = Vina(sf_name='vina')

v.set_receptor('7en8_receptor.pdbqt')

v.set_ligand_from_file(f'{mol_id}.pdbqt')

v.compute_vina_maps(center=[-5.5, -7.75,26.25], box_size=[12.0, 15.5, 18.5])

'''
# Score the current pose
energy = v.score()
print('Score before minimization: %.3f (kcal/mol)' % energy[0])

# Minimized locally the current pose
energy_minimized = v.optimize()
print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
v.write_pose('1iep_ligand_minimized.pdbqt', overwrite=True)
'''

# Dock the ligand
v.dock(exhaustiveness=32, n_poses=10)
v.write_poses(f'{mol_id}_out.pdbqt', n_poses=10, overwrite=True)

Computing Vina grid ... done.
Performing docking (random seed: 1528748622) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -6.791          0          0
   2       -6.653      1.247      1.566
   3       -6.591      1.517      2.524
   4       -6.367      6.179      8.488
   5       -6.292      6.362      8.784
   6       -6.196      1.421      1.519
   7       -6.113      1.337      1.641
   8       -6.024      2.603       3.31
   9        -5.95      3.766      4.663
  10       -5.913      2.255      2.631


In [15]:
# Function to extract binding affinities from Vina log file
def extract_vina_results(file_path):
    affinities = []
    
    # Open the Vina log file for reading
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("REMARK VINA RESULT:"):
                # Split the line by spaces and extract the affinity value (2nd element after splitting)
                affinity = float(line.split()[3])
                affinities.append(affinity)
    
    return affinities[0]

# Example usage
vina_output_file = 'may1_out.pdbqt'  # Replace with the actual path to your file
affinities = extract_vina_results(vina_output_file)

# Output the results
print("Extracted binding affinities:", affinities)


Extracted binding affinities: -6.791
